In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Data Import

In [ ]:
import datatable as dt

# Memory saving function credit to https://www.kaggle.com/gemartin/load-data-reduce-memory-usage
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))

    for col in df.columns:
        col_type = df[col].dtype.name

        if col_type not in ['object', 'category', 'datetime64[ns, UTC]']:
            c_min = df[col].min()
            c_max = df[col].max()
            c_sum = df[col].sum()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_sum < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int16).min and c_sum < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_sum < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_sum < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_sum < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_sum < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))

    return df


In [ ]:
df_train = (
    dt.fread('../input/jane-street-market-prediction/train.csv')
      .to_pandas()
      .query('weight > 0')
      .pipe(reduce_mem_usage)
)

feature_names = df_train.columns[df_train.columns.str.contains('feature')]

# Data Adjustments

In [ ]:
id_field = 'ts_id'
date_field = 'date'
featstr = [i for i in df_train.columns if 'feature_' in i]

df_train['action'] = np.where(df_train['resp'] > 0,1,0)
df_train = df_train[df_train.weight > 0.0]
df_train.fillna(df_train.mean(axis=0), inplace=True)

df_train.action = df_train.action.astype('category')
df_train.feature_0 = df_train.feature_0.astype('category')

In [ ]:
df_train.set_index(['ts_id', 'date'], inplace=True)

target = 'action'
features_list = ['weight'] + [feat for feat in df_train.columns if 'feature_' in str(feat)]
fillna_dict = df_train.mean(axis=0).to_dict()

# Train, Validation & Test Split

In [ ]:
trainval_limit = 300
valtest_limit = 400

X_train = df_train[df_train.index.get_level_values(1) <= trainval_limit][features_list]
y_train = df_train[df_train.index.get_level_values(1) <= trainval_limit][target]

X_val = df_train[(df_train.index.get_level_values(1) > trainval_limit) & (df_train.index.get_level_values(1) <= valtest_limit)][features_list]
y_val = df_train[(df_train.index.get_level_values(1) > trainval_limit) & (df_train.index.get_level_values(1) <= valtest_limit)][target]

X_test = df_train[df_train.index.get_level_values(1) > valtest_limit][features_list]
y_test = df_train[df_train.index.get_level_values(1) > valtest_limit][target]

In [ ]:
# from sklearn.model_selection import train_test_split

# # Reduce the samples size for running faster
# X_train, _, y_train, _ = train_test_split(X_train,
#                                           y_train,
#                                           train_size = 0.5,
#                                           test_size = 0.5,
#                                           stratify = y_train,
#                                           random_state = 27)
# X_val, _, y_val, _ = train_test_split(X_val,
#                                           y_val,
#                                           train_size = 0.5,
#                                           test_size = 0.5,
#                                           stratify = y_val,
#                                           random_state = 27)
# X_test, _, y_test, _ = train_test_split(X_test,
#                                           y_test,
#                                           train_size = 0.5,
#                                           test_size = 0.5,
#                                           stratify = y_test,
#                                           random_state = 27)

In [ ]:
X_train_hyperopt = X_train.copy()
y_train_hyperopt = y_train.copy()
X_test_hyperopt = X_val.copy()
y_test_hyperopt = y_val.copy()

categ_vars = ['feature_0']

# Selected based on positive feature importance
ut_pos_features = ut_pos_features = ['feature_1', 'feature_2', 'feature_3', 'feature_4', 
                   'feature_5', 'feature_6', 'feature_7', 'feature_8', 
                   'feature_10', 'feature_11', 'feature_12', 'feature_14', 
                   'feature_15', 'feature_16', 'feature_17', 'feature_18', 
                   'feature_19', 'feature_20', 'feature_21', 'feature_22', 
                   'feature_23', 'feature_24', 'feature_25', 'feature_26', 
                   'feature_27', 'feature_28', 'feature_29', 'feature_30', 
                   'feature_31', 'feature_32', 'feature_34', 'feature_35', 
                   'feature_36', 'feature_37', 'feature_38', 'feature_39', 
                   'feature_40', 'feature_41', 'feature_42', 'feature_43', 
                   'feature_44', 'feature_45', 'feature_46', 'feature_47', 
                   'feature_48', 'feature_49', 'feature_51', 'feature_52', 
                   'feature_54', 'feature_55', 'feature_56', 'feature_57', 
                   'feature_59', 'feature_60', 'feature_61', 'feature_62', 
                   'feature_63', 'feature_64', 'feature_66', 'feature_68', 
                   'feature_69', 'feature_70', 'feature_71', 'feature_72', 
                   'feature_73', 'feature_74', 'feature_75', 'feature_76', 
                   'feature_77', 'feature_78', 'feature_79', 'feature_82', 
                   'feature_83', 'feature_86', 'feature_87', 'feature_89', 
                   'feature_90', 'feature_91', 'feature_92', 'feature_95', 
                   'feature_96', 'feature_97', 'feature_98', 'feature_99', 
                   'feature_100', 'feature_101', 'feature_102', 'feature_103', 
                   'feature_104', 'feature_105', 'feature_106', 'feature_107', 
                   'feature_108', 'feature_109', 'feature_111', 'feature_112', 
                   'feature_113', 'feature_115', 'feature_116', 'feature_118', 
                   'feature_120', 'feature_121', 'feature_123', 'feature_124', 
                   'feature_125', 'feature_126', 'feature_127', 'feature_129']

# Model Evaluation

In [ ]:
from catboost import CatBoostClassifier
model = CatBoostClassifier(border_count=70, 
                           class_weights=(1, 0.9710117916727243),
                           depth=5,
                           iterations=875,
                           l2_leaf_reg=37.89277395675028,
                           learning_rate=0.0683720060874196,
                           logging_level='Silent',
                           task_type='CPU')

In [ ]:
model.fit(X_train_hyperopt, 
          y_train_hyperopt, 
          cat_features = categ_vars) 

In [ ]:
def utility_function(X, model):
    data = X.copy()
    data = data.reset_index().set_index('ts_id')

    data['action'] = np.round(model.predict_proba(X)[:, 1])
    
    data = data.reset_index().merge(df_train.reset_index()[['ts_id','resp']], how='left', on='ts_id').set_index('ts_id')
    if 'weight' not in list(data.columns):
        data = data.reset_index().merge(df_train.reset_index()[['ts_id','weight']], how='left', on='ts_id').set_index('ts_id')

    data['prod'] = data['weight'] * data['resp'] * data['action']
    data_agg = data.groupby('date')['prod'].sum()

    t = data_agg.sum() / np.sqrt(np.power(data_agg, 2).sum()) * np.sqrt(250 / len(data_agg))

    u = min(max(t, 0), 6) * data_agg.sum()
    
    return u 

# Check
# utility_function(X_test_hyperopt, model)

In [ ]:
def predict_compute_metrics(X, y, model):
    
    predictions = model.predict_proba(X)[:, 1]
    predictions_l = model.predict(X)

    print("Utility function: " + str(np.round(utility_function(X, model), decimals=4)))
    
    return predictions

In [ ]:
pred_val = predict_compute_metrics(X_val, y_val, model)

**Group K-Fold Cross Validation**

In [ ]:
from sklearn.model_selection._split import _BaseKFold, indexable, _num_samples
from sklearn.utils.validation import _deprecate_positional_args

# https://github.com/getgaurav2/scikit-learn/blob/d4a3af5cc9da3a76f0266932644b884c99724c57/sklearn/model_selection/_split.py#L2243
class GroupTimeSeriesSplit(_BaseKFold):
    """Time Series cross-validator variant with non-overlapping groups.
    Provides train/test indices to split time series data samples
    that are observed at fixed time intervals according to a
    third-party provided group.
    In each split, test indices must be higher than before, and thus shuffling
    in cross validator is inappropriate.
    This cross-validation object is a variation of :class:`KFold`.
    In the kth split, it returns first k folds as train set and the
    (k+1)th fold as test set.
    The same group will not appear in two different folds (the number of
    distinct groups has to be at least equal to the number of folds).
    Note that unlike standard cross-validation methods, successive
    training sets are supersets of those that come before them.
    Read more in the :ref:`User Guide <cross_validation>`.
    Parameters
    ----------
    n_splits : int, default=5
        Number of splits. Must be at least 2.
    max_train_size : int, default=None
        Maximum size for a single training set.
    Examples
    --------
    >>> import numpy as np
    >>> from sklearn.model_selection import GroupTimeSeriesSplit
    >>> groups = np.array(['a', 'a', 'a', 'a', 'a', 'a',\
                           'b', 'b', 'b', 'b', 'b',\
                           'c', 'c', 'c', 'c',\
                           'd', 'd', 'd'])
    >>> gtss = GroupTimeSeriesSplit(n_splits=3)
    >>> for train_idx, test_idx in gtss.split(groups, groups=groups):
    ...     print("TRAIN:", train_idx, "TEST:", test_idx)
    ...     print("TRAIN GROUP:", groups[train_idx],\
                  "TEST GROUP:", groups[test_idx])
    TRAIN: [0, 1, 2, 3, 4, 5] TEST: [6, 7, 8, 9, 10]
    TRAIN GROUP: ['a' 'a' 'a' 'a' 'a' 'a']\
    TEST GROUP: ['b' 'b' 'b' 'b' 'b']
    TRAIN: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10] TEST: [11, 12, 13, 14]
    TRAIN GROUP: ['a' 'a' 'a' 'a' 'a' 'a' 'b' 'b' 'b' 'b' 'b']\
    TEST GROUP: ['c' 'c' 'c' 'c']
    TRAIN: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]\
    TEST: [15, 16, 17]
    TRAIN GROUP: ['a' 'a' 'a' 'a' 'a' 'a' 'b' 'b' 'b' 'b' 'b' 'c' 'c' 'c' 'c']\
    TEST GROUP: ['d' 'd' 'd']
    """
    @_deprecate_positional_args
    def __init__(self,
                 n_splits=5,
                 *,
                 max_train_size=None
                 ):
        super().__init__(n_splits, shuffle=False, random_state=None)
        self.max_train_size = max_train_size

    def split(self, X, y=None, groups=None):
        """Generate indices to split data into training and test set.
        Parameters
        ----------
        X : array-like of shape (n_samples, n_features)
            Training data, where n_samples is the number of samples
            and n_features is the number of features.
        y : array-like of shape (n_samples,)
            Always ignored, exists for compatibility.
        groups : array-like of shape (n_samples,)
            Group labels for the samples used while splitting the dataset into
            train/test set.
        Yields
        ------
        train : ndarray
            The training set indices for that split.
        test : ndarray
            The testing set indices for that split.
        """
        if groups is None:
            raise ValueError(
                "The 'groups' parameter should not be None")
        X, y, groups = indexable(X, y, groups)
        n_samples = _num_samples(X)
        n_splits = self.n_splits
        n_folds = n_splits + 1
        group_dict = {}
        u, ind = np.unique(groups, return_index=True)
        unique_groups = u[np.argsort(ind)]
        n_samples = _num_samples(X)
        n_groups = _num_samples(unique_groups)
        for idx in np.arange(n_samples):
            if (groups[idx] in group_dict):
                group_dict[groups[idx]].append(idx)
            else:
                group_dict[groups[idx]] = [idx]
        if n_folds > n_groups:
            raise ValueError(
                ("Cannot have number of folds={0} greater than"
                 " the number of groups={1}").format(n_folds,
                                                     n_groups))
        group_test_size = n_groups // n_folds
        group_test_starts = range(n_groups - n_splits * group_test_size,
                                  n_groups, group_test_size)
        for group_test_start in group_test_starts:
            train_array = []
            test_array = []
            for train_group_idx in unique_groups[:group_test_start]:
                train_array_tmp = group_dict[train_group_idx]
                train_array = np.sort(np.unique(
                                      np.concatenate((train_array,
                                                      train_array_tmp)),
                                      axis=None), axis=None)
            train_end = train_array.size
            if self.max_train_size and self.max_train_size < train_end:
                train_array = train_array[train_end -
                                          self.max_train_size:train_end]
            for test_group_idx in unique_groups[group_test_start:
                                                group_test_start +
                                                group_test_size]:
                test_array_tmp = group_dict[test_group_idx]
                test_array = np.sort(np.unique(
                                              np.concatenate((test_array,
                                                              test_array_tmp)),
                                     axis=None), axis=None)
            yield [int(i) for i in train_array], [int(i) for i in test_array]

In [ ]:
def classification_kfold(X, y, model):
    X_ttv = X.reset_index()[['ts_id','date']]
    X_ttv = X_ttv.sort_values(by=['date','ts_id'])
    
    if 'feature_0' in list(X.columns):
        categ_vars = ['feature_0']
    else:
        categ_vars = []
    
    i = 1
    for train_index, valid_index in GroupTimeSeriesSplit().split(X_ttv, groups=X_ttv['date']):
        print('Fold {}'.format(i))
        train_index_list = X_ttv.loc[train_index, 'date'].unique()
        valid_index_list = X_ttv.loc[valid_index, 'date'].unique()
        print("      [Train dates: " + str(np.min(train_index_list)) + " ---- " + str(np.max(train_index_list)) + "]")
        print("      [Validation dates: " + str(np.min(valid_index_list)) + " ---- " + str(np.max(valid_index_list)) + "]")

        train_cv = X[X.index.get_level_values(1).isin(train_index_list)]
        y_train_cv = y[X.index.get_level_values(1).isin(train_index_list)]
        valid_cv = X[X.index.get_level_values(1).isin(list(valid_index_list))]
        y_valid_cv = y[X.index.get_level_values(1).isin(list(valid_index_list))]

        model.fit(train_cv, 
                  y_train_cv, 
                  # eval_set = (valid_cv, np.ravel(y_valid_cv)), 
                  # early_stopping_rounds=60,
                  cat_features = categ_vars) 

        print("   Train: ")
        pred_trainval = predict_compute_metrics(train_cv, y_train_cv, model)
        print("   Validation: ")
        pred_test = predict_compute_metrics(valid_cv, y_valid_cv, model)
        print('')

        i += 1
        
    return model

In [ ]:
X_trainval = df_train[df_train.index.get_level_values(1) <= valtest_limit][features_list]
y_trainval = df_train[df_train.index.get_level_values(1) <= valtest_limit][target]

model_tv = CatBoostClassifier(border_count=70, 
                             class_weights=(1, 0.9710117916727243),
                             depth=5,
                             iterations=875,
                             l2_leaf_reg=37.89277395675028,
                             learning_rate=0.0683720060874196,
                             logging_level='Silent',
                             task_type='CPU')

In [ ]:
model_tvk_pos = classification_kfold(X_trainval.loc[:,ut_pos_features], y_trainval, model_tv)

In [ ]:
print("Train & validation: ")
pred_train = predict_compute_metrics(X_trainval.loc[:,ut_pos_features], y_trainval, model_tvk_pos)
print("Test: ")
pred_test = predict_compute_metrics(X_test.loc[:,ut_pos_features], y_test, model_tvk_pos)

In [ ]:
import janestreet
from tqdm.notebook import tqdm

env = janestreet.make_env() # initialize the environment
iter_test = env.iter_test() # an iterator which loops over the test set

for (test_df, sample_prediction_df) in tqdm(iter_test):
    sample_prediction_df.action = model_tvk_pos.predict(test_df.loc[:,ut_pos_features]) # make your 0/1 prediction here
    env.predict(sample_prediction_df)

In [ ]:
!rm -r /kaggle/working/catboost_info